In [1]:
!pip3 install protobuf==3.11

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/home/indika/anaconda3/envs/myenv/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorboard

In [3]:
tensorboard.__version__

'2.6.0'

In [4]:
import matplotlib.pyplot as plt
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [6]:
# Version
print(torch.__version__, torchvision.__version__)
data = pd.read_csv("./data.csv")
idx = 0

1.9.1+cu111 0.10.1+cu111


In [7]:
# Loading Data
def load_data(data=data, test=False):
    if test is True:
        if "data.npy" in os.listdir("./"):
            data = np.load("./data.npy", allow_pickle=True)[:500]
            return data
    if "data.npy" in os.listdir("./"):
        data = np.load("./data.npy", allow_pickle=True)
        return data
    new_data = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        xmin, ymin, xmax, ymax = info["xmin"], info["ymin"], info["xmax"], info["ymax"]
        height, width = cv2.imread("./data/" + info["image"]).shape[:2]
        record["file_name"] = "./data/" + info["image"]
        record["height"] = height
        record["width"] = width
        record["cateogry_id"] = 0
        objs = [
            {
                "bbox": [info["xmin"], info["ymin"], info["xmax"], info["ymax"]],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 0,
            }
        ]
        record["image_id"] = idx
        record["annotations"] = objs
        new_data.append(record)
    np.save("data.npy", new_data)
    return new_data

In [8]:
# Config
labels = ["Card"]

In [9]:
# Adding the data
DatasetCatalog.register("data", lambda: load_data())
MetadataCatalog.get("data").set(thing_classes=labels)
metadata = MetadataCatalog.get("data")
DatasetCatalog.register("test", lambda: load_data(test=True))
MetadataCatalog.get("test").set(thing_classes=labels)
metadata_test = MetadataCatalog.get("test")

In [11]:
wandb.init(sync_tensorboard=True,name='baseline')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_C4_3x.yaml'))
cfg.DATASETS.TRAIN = ('data',)
cfg.DATASETS.TEST = ()
cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_C4_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2500
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

wandb: Waiting for W&B process to finish, PID 63686
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20211012_092641-3859g373/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20211012_092641-3859g373/logs/debug-internal.log
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Synced baseline: https://app.wandb.ai/ranuga-d/uncategorized/runs/3859g373
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20211012_092749-jsfehlay
wandb: Syncing run baseline



[10/12 09:27:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (2, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[10/12 09:27:57 d2.engine.train_loop]: Starting training from iteration 0
[10/12 09:28:08 d2.utils.events]:  eta: 0:20:10  iter: 19  total_loss: 1.092  loss_cls: 0.6161  loss_box_reg: 0.4701  loss_rpn_cls: 0.003915  loss_rpn_loc: 0.004176  time: 0.4952  data_time: 0.0089  lr: 4.9953e-06  max_mem: 3364M
[10/12 09:28:17 d2.utils.events]:  eta: 0:20:11  iter: 39  total_loss: 1.002  loss_cls: 0.5534  loss_box_reg: 0.4528  loss_rpn_cls: 0.002985  loss_rpn_loc: 0.006378  time: 0.4921  data_time: 0.0025  lr: 9.9902e-06  max_mem: 3364M
[10/12 09:28:27 d2.utils.events]:  eta: 0:20:00  iter: 59  total_loss: 0.903  loss_cls: 0.4482  loss_box_reg: 0.4261  loss_rpn_cls: 0.006906  loss_rpn_loc: 0.00411  time: 0.4911  data_time: 0.0025  lr: 1.4985e-05  max_mem: 3364M
[10/12 09:28:37 d2.utils.events]:  eta: 0:19:51  iter: 79  total_loss: 0.8562  loss_cls: 0.3592  loss_box_reg: 0.4741  loss_rpn_cls: 0.007636  loss_rpn_loc: 0.004412  time: 0.4896  data_time: 0.0025  lr: 1.998e-05  max_mem: 3364M
[10/12 

In [12]:
evaluator = COCOEvaluator("test", output_dir="./output")
val_loader = build_detection_test_loader(cfg,"test")
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
wandb.log(metrics)

[10/12 09:48:53 d2.evaluation.coco_evaluation]: 'test' is not registered by `register_coco_instances`. Therefore trying to convert it to COCO format ...
[10/12 09:48:53 d2.data.datasets.coco]: Converting annotations of dataset 'test' to COCO format ...)
[10/12 09:48:53 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[10/12 09:48:53 d2.data.datasets.coco]: Conversion finished, #images: 500, #annotations: 500
[10/12 09:48:53 d2.data.datasets.coco]: Caching COCO format annotations at './output/test_coco_format.json' ...
[10/12 09:48:53 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    Card    | 500          |
|            |              |
[10/12 09:48:53 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/12 09:48:53 d2.data.common]: Serializing 500 elements to byte tensors and 

In [13]:
cfg.MODEL.WEIGHTS = "./output/model_final.pth"
predictor = DefaultPredictor(cfg)
cfg.MODEL.WEIGHTS = "./output/model_final.pth"
evaluator = COCOEvaluator("test", output_dir="./output")
val_loader = build_detection_test_loader(cfg,"test")
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
wandb.log(metrics)

[10/12 09:51:06 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/12 09:51:06 d2.data.common]: Serializing 500 elements to byte tensors and concatenating them all ...
[10/12 09:51:06 d2.data.common]: Serialized dataset takes 0.18 MiB
[10/12 09:51:06 d2.evaluation.evaluator]: Start inference on 500 batches
[10/12 09:51:10 d2.evaluation.evaluator]: Inference done 14/500. Dataloading: 0.0008 s/iter. Inference: 0.2596 s/iter. Eval: 0.0001 s/iter. Total: 0.2606 s/iter. ETA=0:02:06
[10/12 09:51:15 d2.evaluation.evaluator]: Inference done 33/500. Dataloading: 0.0010 s/iter. Inference: 0.2638 s/iter. Eval: 0.0001 s/iter. Total: 0.2651 s/iter. ETA=0:02:03
[10/12 09:51:20 d2.evaluation.evaluator]: Inference done 52/500. Dataloading: 0.0010 s/iter. Inference: 0.2643 s/iter. Eval: 0.0001 s/iter. Total: 0.2655 s/iter. ETA=0:01:58
[10/12 09:51:25 d2.evaluation.evaluator]: Inference do

In [14]:
predictor = DefaultPredictor(cfg)
cfg.MODEL.WEIGHTS = "./output/model_final.pth"
evaluator = COCOEvaluator("test", output_dir="./output")
val_loader = build_detection_test_loader(cfg,"test")
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
wandb.log(metrics)

[10/12 09:53:17 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/12 09:53:17 d2.data.common]: Serializing 500 elements to byte tensors and concatenating them all ...
[10/12 09:53:17 d2.data.common]: Serialized dataset takes 0.18 MiB
[10/12 09:53:17 d2.evaluation.evaluator]: Start inference on 500 batches
[10/12 09:53:20 d2.evaluation.evaluator]: Inference done 11/500. Dataloading: 0.0008 s/iter. Inference: 0.2576 s/iter. Eval: 0.0001 s/iter. Total: 0.2586 s/iter. ETA=0:02:06
[10/12 09:53:25 d2.evaluation.evaluator]: Inference done 31/500. Dataloading: 0.0010 s/iter. Inference: 0.2571 s/iter. Eval: 0.0001 s/iter. Total: 0.2583 s/iter. ETA=0:02:01
[10/12 09:53:30 d2.evaluation.evaluator]: Inference done 51/500. Dataloading: 0.0010 s/iter. Inference: 0.2577 s/iter. Eval: 0.0001 s/iter. Total: 0.2589 s/iter. ETA=0:01:56
[10/12 09:53:35 d2.evaluation.evaluator]: Inference do

In [15]:
for img in os.listdir("./test_imgs/"):
    v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
    v = v.draw_instance_predictions(
        predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
    )
    v = v.get_image()[:, :, ::-1]
    plt.figure(figsize=(12, 6))
    plt.imshow(v)
    plt.savefig("./img.png")
    plt.close()
    wandb.log({f"Img/{img}": wandb.Image(cv2.imread("./img.png"))})

FileNotFoundError: [Errno 2] No such file or directory: './test_imgs/'

In [ ]:
# evaluator = COCOEvaluator("data", output_dir="./output")
# val_loader = build_detection_test_loader(cfg,"data")
# metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
# wandb.log(metrics)
# wandb.finish()

In [ ]:
os.environ['TENSORBOARD_BINARY'] = '/path/to/envs/my_env/bin/tensorboard'

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime, os

In [ ]:
%tensorboard --logdir output

In [ ]:
%tensorboard --logdir outputs # TODO : remove this this is wrong.

In [ ]:
!ls

In [ ]:
from tensorboard import notebook

In [ ]:
notebook.list() # View open TensorBoard instances

In [ ]:

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "data")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_C4_3x.yaml'))
cfg.DATASETS.TRAIN = ('data',)
cfg.DATASETS.TEST = ()
cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2500
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "data")
print(inference_on_dataset(predictor.model, val_loader, evaluator))